In [4]:
!pip install ultralytics -q

from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
from IPython.display import Image
import time


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
import zipfile
import os

# Upload 'pests_dataset.zip' via Colab file upload or place in /content
dataset_zip = "/content/pests_dataset.zip"
dataset_dir = "/content/pests_dataset"

# Unzip
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

# Check contents
!ls -R {dataset_dir}


/content/pests_dataset:
pests_dataset

/content/pests_dataset/pests_dataset:
test  test.csv	train  train.csv

/content/pests_dataset/pests_dataset/test:
ants  beetle	    earthworms	grasshopper  slug   wasp
bees  catterpillar  earwig	moth	     snail  weevil

/content/pests_dataset/pests_dataset/test/ants:
'ants (101).jpg'  'ants (221).jpg'  'ants (311).jpg'  'ants (42).jpg'
'ants (102).jpg'  'ants (224).jpg'  'ants (315).jpg'  'ants (430).jpg'
'ants (103).jpg'  'ants (228).jpg'  'ants (317).jpg'  'ants (436).jpg'
'ants (107).jpg'  'ants (229).jpg'  'ants (319).jpg'  'ants (439).jpg'
'ants (108).jpg'  'ants (22).jpg'   'ants (338).jpg'  'ants (440).jpg'
'ants (111).jpg'  'ants (235).jpg'  'ants (339).jpg'  'ants (445).jpg'
'ants (117).jpg'  'ants (237).jpg'  'ants (341).jpg'  'ants (451).jpg'
'ants (121).jpg'  'ants (23).jpg'   'ants (345).jpg'  'ants (454).jpg'
'ants (126).jpg'  'ants (242).jpg'  'ants (34).jpg'   'ants (459).jpg'
'ants (131).jpg'  'ants (248).jpg'  'ants (350).jpg'  'a

In [6]:
import os
import shutil
import random

base_path = "/content/pests_dataset/pests_dataset"
train_path = os.path.join(base_path, "train")
val_path = os.path.join(base_path, "val")

# Create val folder
os.makedirs(val_path, exist_ok=True)

# Split 20% of images into val
for cls in os.listdir(train_path):
    cls_path = os.path.join(train_path, cls)
    cls_val_path = os.path.join(val_path, cls)
    os.makedirs(cls_val_path, exist_ok=True)

    images = os.listdir(cls_path)
    val_count = int(0.2 * len(images))
    val_images = random.sample(images, val_count)

    for img in val_images:
        shutil.move(os.path.join(cls_path, img), os.path.join(cls_val_path, img))

# Check structure
!ls /content/pests_dataset/pests_dataset


test  test.csv	train  train.csv  val


In [7]:
# YOLOv8n (nano) for fast training; switch to yolov8s for better accuracy
model = YOLO("yolov8n-cls.pt")


In [8]:
train_epochs = 20
img_size = 224
batch_size = 32
data_path = "/content/pests_dataset/pests_dataset"  # correct path

for epoch in range(train_epochs):
    start_time = time.time()

    model.train(
        data=data_path,
        epochs=1,       # train one epoch at a time to log ETA
        imgsz=img_size,
        batch=batch_size
    )

    elapsed = time.time() - start_time
    remaining = (train_epochs - epoch - 1) * elapsed
    print(f"Epoch {epoch+1}/{train_epochs} done in {elapsed:.2f}s, "
          f"Estimated time remaining: {remaining/60:.2f} min")


Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/pests_dataset/pests_dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

In [10]:
from google.colab import files
!zip -r pest_outputs.zip runs/classify/train
files.download("pest_outputs.zip")


  adding: runs/classify/train/ (stored 0%)
  adding: runs/classify/train/args.yaml (deflated 53%)
  adding: runs/classify/train/val_batch0_pred.jpg (deflated 1%)
  adding: runs/classify/train/confusion_matrix.png (deflated 18%)
  adding: runs/classify/train/val_batch0_labels.jpg (deflated 1%)
  adding: runs/classify/train/weights/ (stored 0%)
  adding: runs/classify/train/weights/best.pt (deflated 9%)
  adding: runs/classify/train/weights/last.pt (deflated 9%)
  adding: runs/classify/train/results.csv (deflated 38%)
  adding: runs/classify/train/train_batch2.jpg (deflated 4%)
  adding: runs/classify/train/val_batch1_labels.jpg (deflated 1%)
  adding: runs/classify/train/train_batch1.jpg (deflated 3%)
  adding: runs/classify/train/val_batch2_labels.jpg (deflated 1%)
  adding: runs/classify/train/confusion_matrix_normalized.png (deflated 14%)
  adding: runs/classify/train/train_batch0.jpg (deflated 3%)
  adding: runs/classify/train/val_batch2_pred.jpg (deflated 1%)
  adding: runs/classif

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# # Resume from last checkpoint (optional)
# model = YOLO("runs/classify/train/weights/last.pt")
# model.train(resume=True, epochs=30)  # continue training more epochs
